In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re

# step2.로그인 정보 및 검색할 회사 미리 정의, 해당 회사의 리뷰 끝 페이지도 정의
usr = "20180908@sungshin.ac.kr"
pwd = "xkatktnchlrh!"


# step3.크롬드라이버 실행 및 잡플래닛 로그인

driver=webdriver.Chrome('driver/chromedriver')

driver.get("https://www.jobplanet.co.kr") #https://www.jobplanet.co.kr/companies?&
time.sleep(5)
login=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > div.right_box > div.login_before.show > a.btn_txt.login > span")
login.click()

login_id = driver.find_element_by_css_selector("input#user_email")
login_id.send_keys(usr)

login_pwd = driver.find_element_by_css_selector("input#user_password")

login_pwd.send_keys(pwd)

login_id.send_keys(Keys.RETURN)
time.sleep(5)


rank=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > nav > ul > li.companies.btn_global_header > a")
rank.click()
time.sleep(3)

rank2=driver.find_element_by_css_selector("body > div.body_wrap > div.globalHeaderWrap > header > div.global_header_modal.global_header_open > div.header_modal_wrap > div > div.header_modal_left > div.modal_left_top > div > a > span")
rank2.click()
time.sleep(5)
# :nth-child 안 숫자 조정해서 산업군 별로
driver.find_element_by_css_selector("#navIndustrySide > div > div.side_body > ul > li:nth-child(6) > a > span").click()
# 산업군 선택 페이지 저장
currnet_url = driver.current_url



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"body > div.body_wrap > div.globalHeaderWrap > header > div.global_header_modal.global_header_open > div.header_modal_wrap > div > div.header_modal_left > div.modal_left_top > div > a > span"}
  (Session info: chrome=92.0.4515.107)


In [8]:
### 날짜랑 직무 추가 크롤링 ###
list_div = []
list_cur = []
list_date =[]
list_stars = []
list_summery = []
list_merit = []
list_disadvantages = []
list_managers =[]
list_name=[]
rate_list=[]

for i in range(5, 6): #range(1,11)
    cur_page=1
    driver.find_element_by_css_selector("#listCompanies > div > div.section_group > section:nth-child({}) > div > div > dl.content_col2_3.cominfo > dt > a".format(i)).click()
    #회사 페이지 들어가기
    time.sleep(10)
    #리뷰 페이지 클릭
    #driver.find_element_by_css_selector('#viewCompaniesMenu > ul > li.viewReviews > a').click()
    time.sleep(5)
    #프리미엄 구매 버튼 체크
    try:
        premium = driver.find_element_by_css_selector('#premiumReviewChart > div > div.layer_popup_box.layer_popup_box_on > div.layer_popup.jply_modal.premium_review_inform > div > div.premium_modal_header > button')
        premium.click()
        time.sleep(7)
    except:
        pass
    #회사 이름 저장
    name=driver.find_element_by_css_selector("body > div.body_wrap > div.cmp_hd > div.new_top_bnr > div > div.top_bnr_wrap > div > div > div.company_info_sec > div.company_info_box > div.company_name > h1 > a").text
    #전체 리뷰 통계 텍스트
    #rate_list.append([name]+[x for x in driver.find_element_by_css_selector("#premiumReviewStatistics > div > div > div").text.split("\n") if re.search("\d",x)])
    #맨 뒤 페이지 클릭
    driver.find_element_by_css_selector("#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > a.btn_pglast").click()
    time.sleep(7)
    #맨 뒤 페이지 숫자 저장
    page=driver.find_element_by_css_selector("#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > strong").text
    time.sleep(3)
    driver.back() #다시 처음으로 
    time.sleep(5)
    
    
    for i in range(0, int(page)): #페이지 수대로 for문

            # css_selector가 span.txt1인 element가 로딩될 때 까지 10초 대기
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.txt1")))
        except TimeoutException:
            # 실패 시에는 에러메시지로 Time Out 출력
            print('Time Out')

        user_info = driver.find_elements_by_css_selector("div.content_top_ty2")
        count = len(user_info)
        cur_page+=1
            #직무, 근속여부, 일시
        list_user_info = []

        for j in user_info:
            list_user_info.append(j.text.split('\n'))

        for j in range(count):  #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
            list_div.append(list_user_info[j][0])
            list_date.append(list_user_info[j][-1])
            list_name.append(name)
        
        summery = driver.find_elements_by_css_selector("h2.us_label")

        for j in summery:
            list_summery.append(j.text)
            #장점, 단점, 경영진에게 바라는 점
        
        list_review = []

        review = driver.find_elements_by_css_selector("dd.df1")
        
        for j in review:
            list_review.append(j.text)

        for j in range(count):      #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
            a = list_review[3*j]
            list_merit.append(a)

            b = list_review[3*j+1]
            list_disadvantages.append(b)

            c = list_review[3*j+2]
            list_managers.append(c)
        
        print(i, '페이지 완료!')
        
            # 다음 페이지 클릭 후 for문 진행, 끝 페이지에서 다음 페이지 클릭 안되는 것 대비해서 예외처리 구문 추가
        try:
            driver.find_element_by_css_selector("#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > a.btn_pgnext").click()
            time.sleep(12)

        except:
            print("다음 페이지 오류")
            pass


    driver.get('https://www.jobplanet.co.kr/companies?industry_id=600') #https://www.jobplanet.co.kr/companies?&

    time.sleep(15)



0 페이지 완료!
1 페이지 완료!
2 페이지 완료!
3 페이지 완료!
4 페이지 완료!
5 페이지 완료!
6 페이지 완료!
7 페이지 완료!
8 페이지 완료!
9 페이지 완료!
10 페이지 완료!
11 페이지 완료!
12 페이지 완료!
13 페이지 완료!
14 페이지 완료!
15 페이지 완료!
16 페이지 완료!
17 페이지 완료!
18 페이지 완료!
19 페이지 완료!
20 페이지 완료!
21 페이지 완료!
22 페이지 완료!
23 페이지 완료!
24 페이지 완료!
25 페이지 완료!
26 페이지 완료!
27 페이지 완료!
28 페이지 완료!
29 페이지 완료!
30 페이지 완료!
31 페이지 완료!
32 페이지 완료!
33 페이지 완료!
34 페이지 완료!
35 페이지 완료!
36 페이지 완료!
37 페이지 완료!
38 페이지 완료!
39 페이지 완료!
40 페이지 완료!
41 페이지 완료!
42 페이지 완료!
43 페이지 완료!
44 페이지 완료!
45 페이지 완료!
46 페이지 완료!
47 페이지 완료!
48 페이지 완료!
49 페이지 완료!
50 페이지 완료!
51 페이지 완료!
52 페이지 완료!
53 페이지 완료!
54 페이지 완료!
55 페이지 완료!
56 페이지 완료!
57 페이지 완료!
58 페이지 완료!
59 페이지 완료!
60 페이지 완료!
61 페이지 완료!
62 페이지 완료!
63 페이지 완료!
64 페이지 완료!
65 페이지 완료!
66 페이지 완료!
67 페이지 완료!
68 페이지 완료!
69 페이지 완료!
70 페이지 완료!
71 페이지 완료!
72 페이지 완료!
73 페이지 완료!
74 페이지 완료!
75 페이지 완료!
76 페이지 완료!
77 페이지 완료!
78 페이지 완료!
79 페이지 완료!
80 페이지 완료!
81 페이지 완료!
82 페이지 완료!
83 페이지 완료!
84 페이지 완료!
85 페이지 완료!
86 페이지 완료!
87 페이지 완료!
88 페이지 완료!
89 페이지 완료!
90 페이지 완료!
91 페이지 완료

ProtocolError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [9]:
total_data = pd.DataFrame()
total_data['회사'] = pd.Series(list_name)
total_data['날짜'] = pd.Series(list_date)
total_data['직무'] = pd.Series(list_div)
total_data['요약'] = pd.Series(list_summery)
total_data['장점'] = pd.Series(list_merit)
total_data['단점'] = pd.Series(list_disadvantages)
total_data['경영진에게 바라는 점'] = pd.Series(list_managers)
total_rate = pd.DataFrame(rate_list,columns=['회사','통계','전체 평점','복지 및 급여','업무와 삶의 균형','사내문화','승진 기회 및 가능성','경영진','기업 추천율','CEO 지지율','성장가능성'])

# step9.엑셀 형태로 저장하기
total_data.to_excel("LGCNS.xls" ,index=True)

# step10.크롬 드라이버 종료
driver.close()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=49963): Max retries exceeded with url: /session/d7a30571ceaddd9aaaa62d8e078b4a54/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f971f1a4610>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [60]:
### 날짜랑 직무 추가 크롤링 ###
for i in range(95, int(page)): #페이지 수대로 for문

            # css_selector가 span.txt1인 element가 로딩될 때 까지 10초 대기
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.txt1")))
    except TimeoutException:
            # 실패 시에는 에러메시지로 Time Out 출력
        print('Time Out')

    user_info = driver.find_elements_by_css_selector("div.content_top_ty2")
    count = len(user_info)
    cur_page+=1
            #직무, 근속여부, 일시
    list_user_info = []

    for j in user_info:
        list_user_info.append(j.text.split('\n'))

    for j in range(count):  #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
        list_div.append(list_user_info[j][0])
        list_date.append(list_user_info[j][-1])
        list_name.append(name)
        
    summery = driver.find_elements_by_css_selector("h2.us_label")

    for j in summery:
        list_summery.append(j.text)
            #장점, 단점, 경영진에게 바라는 점
        
    list_review = []

    review = driver.find_elements_by_css_selector("dd.df1")
        
    for j in review:
        list_review.append(j.text)

    for j in range(count):      #한 페이지에 정보 5set씩 나옴. 마지막 페이지는 5개 미만일 수 있으므로 count 변수를 반복횟수로 넣어줌.
        a = list_review[3*j]
        list_merit.append(a)

        b = list_review[3*j+1]
        list_disadvantages.append(b)

        c = list_review[3*j+2]
        list_managers.append(c)

        
    print(i, '페이지 완료!')
        
            # 다음 페이지 클릭 후 for문 진행, 끝 페이지에서 다음 페이지 클릭 안되는 것 대비해서 예외처리 구문 추가
    try:
        driver.find_element_by_css_selector("#viewReviewsList > div > div > div > div.pg_bottom.um_paginnation > article > a.btn_pgnext").click()
        time.sleep(12)

    except:
        print("다음 페이지 오류")
        pass


driver.get('https://www.jobplanet.co.kr/companies?industry_id=500') #https://www.jobplanet.co.kr/companies?&

time.sleep(15)



95 페이지 완료!
96 페이지 완료!
97 페이지 완료!
98 페이지 완료!
99 페이지 완료!
100 페이지 완료!
101 페이지 완료!
102 페이지 완료!
103 페이지 완료!
104 페이지 완료!
105 페이지 완료!
106 페이지 완료!
107 페이지 완료!
108 페이지 완료!
109 페이지 완료!
110 페이지 완료!
111 페이지 완료!
112 페이지 완료!
113 페이지 완료!
114 페이지 완료!
115 페이지 완료!
116 페이지 완료!
117 페이지 완료!
118 페이지 완료!
119 페이지 완료!
120 페이지 완료!
121 페이지 완료!
122 페이지 완료!
123 페이지 완료!
124 페이지 완료!
125 페이지 완료!
126 페이지 완료!
127 페이지 완료!
128 페이지 완료!
129 페이지 완료!
130 페이지 완료!
131 페이지 완료!
132 페이지 완료!
133 페이지 완료!
134 페이지 완료!
135 페이지 완료!
136 페이지 완료!
137 페이지 완료!
138 페이지 완료!
139 페이지 완료!
140 페이지 완료!
141 페이지 완료!
142 페이지 완료!
143 페이지 완료!
144 페이지 완료!
145 페이지 완료!
146 페이지 완료!
147 페이지 완료!
148 페이지 완료!
149 페이지 완료!
150 페이지 완료!
151 페이지 완료!
152 페이지 완료!
153 페이지 완료!
154 페이지 완료!
155 페이지 완료!
156 페이지 완료!
157 페이지 완료!
158 페이지 완료!
159 페이지 완료!
160 페이지 완료!
161 페이지 완료!
162 페이지 완료!
163 페이지 완료!
164 페이지 완료!
165 페이지 완료!
